# Setup
Install basic dependenciesx

In [1]:
import pandas as pd
import numpy as np
from IPython.display import Markdown, display


# Display settings for pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)


## Settings

How many students per team?

In [2]:
STUDENTS_PER_TEAM = 5

Input and output data filenames

In [3]:
INPUT_FILE = 'data/project_preferences_fall2025.csv'
OUTPUT_FILE = 'data/project_assignments.csv'

## Load input data

A blank dataframe that will hold each student's assigned project once done

In [4]:
df_assignments = pd.DataFrame(columns=['email', 'project', 'own'])

Load the input data file into a dataframe

In [5]:
df_votes = pd.read_csv(INPUT_FILE)


Simplify column names and remove unnecessary columns.

In [6]:
df_votes = df_votes.rename(columns={
    'Email Address': 'email',
    'First choice': 'first',
    'Second choice': 'second',
    'Third choice': 'third',
    'Are any of these choices your own project proposal?': 'voted_for_self'
})[['email', 'first', 'second', 'third', 'voted_for_self']]

Remove all before a dash and trim whitespace... this is specific to how the project names are followed by their descriptions... remove or update as needed.

In [7]:
for col in ['first', 'second', 'third']:
    df_votes[col] = df_votes[col].str.split('-').str[0].str.strip()

Convert all project names to lowercase.

In [8]:
for col in ['first', 'second', 'third']:
    df_votes[col] = df_votes[col].str.lower()

Take a gander.

In [9]:
df_votes.sample(5)

,email,first,second,third,voted_for_self
118,ahc1234@yourdomain.com,project crux,musi,nourishpet,Yes - First choice
22,cp1234@yourdomain.com,gesturetalk,roomiehub,engage 2.0,Yes - First choice
92,sk12349@yourdomain.com,taskbank,nutrismart,rirf,Yes - Second choice
102,ajh773@yourdomain.com,stashly,nourishpet,profpick,No - I did not select my own project proposal ...
76,cc1234@yourdomain.com,instaskill,over budget,strideai,Yes - Second choice


## Determine project names

In [10]:
project_names = pd.Series(pd.unique(df_votes[['first', 'second', 'third']].values.ravel()))

In [11]:
project_names = project_names.str.lower()

In [12]:
project_names.describe()

count             52
unique            52
top       rendezvous
freq               1
dtype: object

In [13]:
project_names.sample(3)

9     pocketbudget
7       cartranker
18     tickerpickr
dtype: object

## Count votes

Total number of first, second, and third choice votes for each project.

In [14]:
def get_remaining_vote_counts(df_votes):
    vote_counts = pd.DataFrame({
        'project': project_names,
        'first_count': [df_votes['first'].str.lower().eq(name).sum() for name in project_names],
        'second_count': [df_votes['second'].str.lower().eq(name).sum() for name in project_names],
        'third_count': [df_votes['third'].str.lower().eq(name).sum() for name in project_names]
    })
    vote_counts['total'] = vote_counts['first_count'] + vote_counts['second_count'] + vote_counts['third_count']

    vote_counts.sort_values(
        by=['first_count', 'second_count', 'third_count'],
        ascending=[False, False, False]
    )
    return vote_counts

In [15]:
vote_counts = get_remaining_vote_counts(df_votes)
vote_counts

,project,first_count,second_count,third_count,total
0,rendezvous,4,1,7,12
1,tbd,0,2,2,4
2,next quad,4,4,0,8
3,kitchen helper,1,0,1,2
4,commyounity,3,2,5,10
5,study space finder @ nyu,3,6,4,13
6,careconnect,2,1,1,4
7,cartranker,1,3,3,7
8,cardwise automations,2,5,2,9
9,pocketbudget,3,3,2,8


Identify projects with insufficient votes

In [16]:
# Identify projects with insufficient total votes
vote_counts = get_remaining_vote_counts(df_votes)
insufficient_projects = vote_counts[vote_counts['total'] < STUDENTS_PER_TEAM][['project', 'total']]
insufficient_projects


,project,total
1,tbd,4
3,kitchen helper,2
6,careconnect,4
19,fishdex,3
30,campusconnect,3
31,spin!,4
33,engage 2.0,3
37,sanpvite,2
43,over budget,3
44,flavors & fortunes,1


Remove those projects with insufficient votes from the vote counts.

In [17]:

# Remove these projects from vote_counts
vote_counts = vote_counts[~vote_counts['project'].isin(insufficient_projects['project'])].reset_index(drop=True)
vote_counts

,project,first_count,second_count,third_count,total
0,rendezvous,4,1,7,12
1,next quad,4,4,0,8
2,commyounity,3,2,5,10
3,study space finder @ nyu,3,6,4,13
4,cartranker,1,3,3,7
5,cardwise automations,2,5,2,9
6,pocketbudget,3,3,2,8
7,tasklatte,2,2,7,11
8,roulette go,3,2,1,6
9,moodsphere,5,2,1,8


Remove mention of those projects from df_votes.

In [18]:
for project in insufficient_projects:
    for col in ['first', 'second', 'third']:
        df_votes.loc[df_votes[col] == project, col] = np.nan

df_votes.sample(10)


,email,first,second,third,voted_for_self
4,sw1234@yourdomain.com,cardwise automations,pocketbudget,tasklatte,Yes - First choice
22,cp1234@yourdomain.com,gesturetalk,roomiehub,engage 2.0,Yes - First choice
115,sm12344@yourdomain.com,nourishpet,gymflow nyu,kitchen helper,Yes - First choice
17,sad1234@yourdomain.com,swapbay,camp,gymflow nyu,Yes - First choice
74,yl12347@yourdomain.com,study space finder @ nyu,dinewise,campus resource finder,Yes - First choice
24,sd1234@yourdomain.com,cash me if you can,dungeons and distractions,toberead,No - I did not select my own project proposal ...
34,jf1234@yourdomain.com,gesturetalk,einstein,upstore,Yes - First choice
111,am12347@yourdomain.com,upstore,swapbay,instaskill,Yes - First choice
2,xs1234@yourdomain.com,kitchen helper,commyounity,study space finder @ nyu,No - I did not select my own project proposal ...
23,mds831@yourdomain.com,fishdex,roomiehub,project crux,Yes - First choice


## New method

In [19]:
def get_project_votes(df_votes, project):
    """
    Returns a dataframe of all rows where the first, second, or third choices were the given project.
    Orders results according to vote priority.    

    """
    df = df_votes.copy()
    df['sort_priority'] = np.select(
        [
            # first choice and wrote it themselves
            (df['first'] == project) & (df['voted_for_self'] == 'Yes - First choice'),
            # first choice but didn't write it
            (df['first'] == project) & (df['voted_for_self'] != 'Yes - First choice'),
            # second choice and wrote it themselves
            (df['second'] == project) & (df['voted_for_self'] == 'Yes - Second choice'),
            # second choice but didn't write it
            (df['second'] == project) & (df['voted_for_self'] != 'Yes - Second choice'),
            # third choice and wrote it themselves
            (df['third'] == project) & (df['voted_for_self'] == 'Yes - Third choice'),
            # third choice but didn't write it
            (df['third'] == project) & (df['voted_for_self'] != 'Yes - Third choice')
        ],
        [6, 5, 4, 3, 2, 1],
        default=0
    )
    mask = (
        (df['first'] == project) |
        (df['second'] == project) |
        (df['third'] == project)
    )
    return df[mask].sort_values(by='sort_priority', ascending=False).reset_index(drop=True)

In [20]:
# a function to check whether a team is full
def is_team_full(df_assignments, project):
    """
    Check if the given project team in df_assignments has reached the maximum number of students.
    Returns True if the team is full, otherwise False.
    """
    return df_assignments[df_assignments['project'] == project].shape[0] >= STUDENTS_PER_TEAM


In [21]:
# Assign students to projects based on their votes and self-proposal status

assigned_emails = set(df_assignments['email'])
projects = vote_counts['project'].tolist()

for priority in range(6):
    for project in projects:
        if is_team_full(df_assignments, project):
            continue
        # Select students based on priority
        if priority == 0:
            mask = (
                (df_votes['first'] == project) &
                (df_votes['voted_for_self'] == 'Yes - First choice')
            )
        elif priority == 1:
            mask = (
                (df_votes['first'] == project) &
                (df_votes['voted_for_self'] != 'Yes - First choice')
            )
        elif priority == 2:
            mask = (
                (df_votes['second'] == project) &
                (df_votes['voted_for_self'] == 'Yes - Second choice')
            )
        elif priority == 3:
            mask = (
                (df_votes['second'] == project) &
                (df_votes['voted_for_self'] != 'Yes - Second choice')
            )
        elif priority == 4:
            mask = (
                (df_votes['third'] == project) &
                (df_votes['voted_for_self'] == 'Yes - Third choice')
            )
        elif priority == 5:
            mask = (
                (df_votes['third'] == project) &
                (df_votes['voted_for_self'] != 'Yes - Third choice')
            )
        candidates = df_votes[mask & (~df_votes['email'].isin(assigned_emails))]
        for _, row in candidates.iterrows():
            if is_team_full(df_assignments, project):
                break
            df_assignments.loc[len(df_assignments)] = {
                'email': row['email'],
                'project': project,
                'own': (
                    (priority == 0) or
                    (priority == 2) or
                    (priority == 4)
                )
            }
            assigned_emails.add(row['email'])

In [22]:

# Function to redistribute remaining votes
df_assignments.sort_values(by=['project', 'own', 'email'])

,email,project,own
33,bb1234@yourdomain.com,agrilink,True
31,bg1234@yourdomain.com,agrilink,True
32,gmo1234@yourdomain.com,agrilink,True
44,ak12347@yourdomain.com,camp,True
43,asm1234@yourdomain.com,camp,True
...,...,...,...
89,xl1234@yourdomain.com,toberead,False
61,am12347@yourdomain.com,upstore,True
59,so1234@yourdomain.com,upstore,True
60,ss12346@yourdomain.com,upstore,True


## Eliminate teams with too few members

All teams by number of members

In [23]:
team_counts = df_assignments['project'].value_counts().sort_values(ascending=False)
print(team_counts)

project
swapbay                      5
tickerpickr                  5
moodsphere                   5
nourishpet                   5
gesturetalk                  5
commyounity                  5
musi                         5
camp                         5
roomiehub                    5
taskbank                     5
instaskill                   4
next quad                    4
profpick                     4
cash me if you can           4
einstein                     3
strideai                     3
nutrismart                   3
study space finder @ nyu     3
cardwise automations         3
agrilink                     3
pocketbudget                 3
upstore                      3
tasklatte                    3
roulette go                  3
toberead                     2
rendezvous                   2
project crux                 2
the language café            2
cartranker                   2
rirf                         1
dungeons and distractions    1
supermix                     1


Too few votes

In [24]:
insufficient_projects = team_counts[team_counts < STUDENTS_PER_TEAM]
insufficient_projects

project
instaskill                   4
next quad                    4
profpick                     4
cash me if you can           4
einstein                     3
strideai                     3
nutrismart                   3
study space finder @ nyu     3
cardwise automations         3
agrilink                     3
pocketbudget                 3
upstore                      3
tasklatte                    3
roulette go                  3
toberead                     2
rendezvous                   2
project crux                 2
the language café            2
cartranker                   2
rirf                         1
dungeons and distractions    1
supermix                     1
dinewise                     1
vodtracker                   1
Name: count, dtype: int64

In [25]:
# Replace project names in df_assignments that are in insufficient_projects with 'unassigned'
df_assignments.loc[df_assignments['project'].isin(insufficient_projects.index), 'project'] = np.nan
df_assignments

,email,project,own
0,nb1234@yourdomain.com,NaN,True
1,dhh1234@yourdomain.com,NaN,True
2,cb1234@yourdomain.com,NaN,True
3,ns1234@yourdomain.com,NaN,True
4,zs1234@yourdomain.com,NaN,True
...,...,...,...
106,nu1234@yourdomain.com,NaN,False
107,jl12347@yourdomain.com,musi,False
108,yl12349@yourdomain.com,roomiehub,False
109,yd1234@yourdomain.com,NaN,False


In [26]:
team_counts = df_assignments['project'].value_counts().sort_values(ascending=False)
print(team_counts)

project
commyounity    5
moodsphere     5
gesturetalk    5
nourishpet     5
tickerpickr    5
swapbay        5
camp           5
roomiehub      5
taskbank       5
musi           5
Name: count, dtype: int64


In [27]:
# Find rows with null project
null_project_mask = df_assignments['project'].isnull()
num_nulls = null_project_mask.sum()
num_nulls

np.int64(61)

In [28]:
import math

num_groups = math.ceil(num_nulls / STUDENTS_PER_TEAM)
num_groups

13

Generate random team names for those unassigned so far

In [29]:
import nltk
import random

nltk.download('words')
from nltk.corpus import words

word_list = [w for w in words.words() if len(w) > 4]
random_names = [
    "random_" + random.choice(word_list).lower()
    for _ in range(num_groups)
]
random_names

[nltk_data] Downloading package words to
[nltk_data]     /Users/foobarstein/nltk_data...
[nltk_data]   Package words is already up-to-date!


['random_undereye',
 'random_pasteurize',
 'random_shillet',
 'random_bioplastic',
 'random_chloranthaceous',
 'random_skiddingly',
 'random_discommendableness',
 'random_mabel',
 'random_carquaise',
 'random_newfangledness',
 'random_uncome',
 'random_overwasted',
 'random_tirthankara']

Assign the random team names to those students who are unassigned.

In [30]:
# Assign random project names in groups
null_indices = df_assignments[null_project_mask].index.tolist()
for i, idx in enumerate(null_indices):
    group_num = i // STUDENTS_PER_TEAM
    df_assignments.at[idx, 'project'] = random_names[group_num]
df_assignments

,email,project,own
0,nb1234@yourdomain.com,random_undereye,True
1,dhh1234@yourdomain.com,random_undereye,True
2,cb1234@yourdomain.com,random_undereye,True
3,ns1234@yourdomain.com,random_undereye,True
4,zs1234@yourdomain.com,random_undereye,True
...,...,...,...
106,nu1234@yourdomain.com,random_overwasted,False
107,jl12347@yourdomain.com,musi,False
108,yl12349@yourdomain.com,roomiehub,False
109,yd1234@yourdomain.com,random_overwasted,False


In [31]:
# Display the number of students assigned to each team, sorted descending
df_assignments['project'].value_counts().sort_values(ascending=False)

project
random_undereye              5
random_pasteurize            5
commyounity                  5
random_shillet               5
moodsphere                   5
gesturetalk                  5
random_bioplastic            5
nourishpet                   5
tickerpickr                  5
random_chloranthaceous       5
swapbay                      5
camp                         5
roomiehub                    5
random_skiddingly            5
taskbank                     5
random_discommendableness    5
musi                         5
random_mabel                 5
random_carquaise             5
random_newfangledness        5
random_uncome                5
random_overwasted            5
random_tirthankara           1
Name: count, dtype: int64

## Save assignments
Save to CSV output data file.

In [32]:
df_assignments = df_assignments.sort_values(by=['project', 'own', 'email'], ascending=[True, False, True])
df_assignments.to_csv(OUTPUT_FILE, index=False)